This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/optimization.html).  I recommend having both windows open, side-by-side!


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown, display
from pydrake.all import MathematicalProgram, Solve, ToLatex

# SDP relaxation of non-convex quadratic constraints

Consider the problem:

$$\begin{aligned} \min_x \quad & \| x - a \|^2 \\ \text{subject to} \quad & \| x - b \| \ge 1 \end{aligned}$$

We can write this as

$$\begin{aligned} \min_{x,y} \quad & y - 2ax + a^2 \\ \text{subject to} \quad & y - 2bx + b^2 \ge 1 \\ & y \ge x^2 \end{aligned}$$

where we write $y \ge x^2$ as the semidefinite constraint
$\begin{bmatrix} y & x \\ x & 1 \end{bmatrix}.$  I've plotted the feasible
region and the objective with an arrow.  As you know, for linear
objectives, the optimal solution will lie on the boundary only if the cost is
directly orthogonal to the objective; otherwise it will lie at a vertex.  So in
this case, the solution will only lie on the interior if $a = b;$ for every
other value, this relaxation will give the optimal solution.  (Try changing $a$ to see).

In [ ]:
def sdp_nonconvex_quadratic():
    fig, ax = plt.subplots()
    a = 0.8
    b = 0.5
    x = np.linspace(-2, 2)
    plt.fill(x, np.maximum(x * x, 1 + 2 * b * x - b * b), color="lightgray")
    ax.plot(x, x * x, "k")
    ax.plot(x, 1 + 2 * b * x - b * b, "k")
    qx = 0
    qy = 3
    ax.quiver(qx, qy, 2 * a, -1, scale=10, zorder=2, color="b")

    ax.set_xlim(-2, 2)
    ax.set_ylim(0, 4)
    ax.set_xlabel("x")
    ax.set_ylabel("y")

    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(1, "x")[0]
    y = prog.NewContinuousVariables(1, "y")[0]

    prog.AddPositiveSemidefiniteConstraint(np.array([[y, x], [x, 1]]))
    prog.AddLinearConstraint(y - 2 * b * x + b * b >= 1)
    prog.AddLinearCost(y - 2 * a * x + a * a)

    result = Solve(prog)
    ax.plot(result.GetSolution(x), result.GetSolution(y), "b*", markersize=15)
    display(plt.show())


sdp_nonconvex_quadratic()

In [ ]:
def sdp_unit_circle():
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(2, "x")
    Y = prog.NewSymmetricContinuousVariables(2, "Y")

    P = np.block([[Y, x.reshape((2, 1))], [x.reshape((1, 2)), 1]])
    display(Markdown("$" + ToLatex(P, precision=2) + " \succeq 0$"))
    prog.AddPositiveSemidefiniteConstraint(P)
    prog.AddLinearConstraint(np.trace(Y) == 1)

    A = np.eye(2)
    b = np.array([0.2, 0.2])
    prog.AddLinearCost(np.trace(A @ Y) + b.dot(x))

    result = Solve(prog)
    print(f"x = {result.GetSolution(x)}")


sdp_unit_circle()

Note that the same program, written with the simpler relaxation ($x^Tx \le 1$), would not be tight, since $$\frac{\partial x^TAx + b^Tx}{\partial x} = 2x^T A + b^T = 0 \Rightarrow x^* = -\frac{1}{2}A^{-1}b = \begin{bmatrix} 0.1 \\ 0.1 \end{bmatrix}$$ is inside the unit circle.

What's really amazing is that the SDP relaxation even works when the objective is concave (instead of convex)!

In [ ]:
def sdp_unit_circle_concave_objective():
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(2, "x")
    Y = prog.NewSymmetricContinuousVariables(2, "Y")

    P = np.block([[Y, x.reshape((2, 1))], [x.reshape((1, 2)), 1]])
    display(Markdown("$" + ToLatex(P, precision=2) + " \succeq 0$"))
    prog.AddPositiveSemidefiniteConstraint(P)
    prog.AddLinearConstraint(np.trace(Y) == 1)

    A = -np.eye(2)
    b = np.array([0.2, 0.2])
    prog.AddLinearCost(np.trace(A @ Y) + b.dot(x))

    result = Solve(prog)
    print(f"x = {result.GetSolution(x)}")


sdp_unit_circle_concave_objective()